## Proposta
    Tomar dados do site da câmara dos deputados e verificar qual o índice de convergência de votos entre os membros de partidos distintos que formaram coalisões

## Inicialização e limpeza dos dados

### Imports

In [1]:
import pandas as pd
import numpy as np

### Inicialização do DataFrame

In [2]:
df = pd.read_csv('votacoesVotos-2021.csv', sep=";", engine="python")

### Verificação das informações do DataFrame

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321494 entries, 0 to 321493
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   idVotacao               321494 non-null  object
 1   uriVotacao              321494 non-null  object
 2   dataHoraVoto            321494 non-null  object
 3   voto                    321146 non-null  object
 4   deputado_id             321494 non-null  int64 
 5   deputado_uri            321494 non-null  object
 6   deputado_nome           321494 non-null  object
 7   deputado_siglaPartido   321494 non-null  object
 8   deputado_uriPartido     321494 non-null  object
 9   deputado_siglaUf        321494 non-null  object
 10  deputado_idLegislatura  321494 non-null  int64 
 11  deputado_urlFoto        321494 non-null  object
dtypes: int64(2), object(10)
memory usage: 29.4+ MB


### Retirada das colunas que não importam nesse caso

In [4]:
df = df.drop(columns=["uriVotacao", "dataHoraVoto", "deputado_id", "deputado_uri", "deputado_nome",
             "deputado_uriPartido", "deputado_siglaUf", "deputado_idLegislatura", "deputado_urlFoto"])

df.head()

,idVotacao,voto,deputado_siglaPartido
0,105464-262,Não,PSL
1,105464-262,Não,REPUBLICANOS
2,105464-262,Não,PSL
3,105464-262,Não,PSL
4,105464-262,Não,PSL


### Conversão de SIM para 1 e NÃO para 0

In [5]:
df.voto = df.voto.eq('Sim').mul(1)
df.head()

,idVotacao,voto,deputado_siglaPartido
0,105464-262,0,PSL
1,105464-262,0,REPUBLICANOS
2,105464-262,0,PSL
3,105464-262,0,PSL
4,105464-262,0,PSL


### Conversão de partidos para números (dados categórigos)

In [6]:
codes_deputado_siglaPartido, uniques_deputado_siglaPartido = pd.factorize(df.deputado_siglaPartido)

df.deputado_siglaPartido = codes_deputado_siglaPartido
df.head()

,idVotacao,voto,deputado_siglaPartido
0,105464-262,0,0
1,105464-262,0,1
2,105464-262,0,0
3,105464-262,0,0
4,105464-262,0,0


### Conversão de id de votação para números (dados categóricos) 

In [7]:
codes_idVotacao, uniques_idVotacao = pd.factorize(df.idVotacao)
df.idVotacao = codes_idVotacao

df.head()

,idVotacao,voto,deputado_siglaPartido
0,0,0,0
1,0,0,1
2,0,0,0
3,0,0,0
4,0,0,0


### Organização da tabela segundo o o id da votação, a sigla do partido

In [8]:
df = df.sort_values(by=["idVotacao", "deputado_siglaPartido"])

sp = df.pop('deputado_siglaPartido')
df.insert(1, 'deputado_siglaPartido', sp)

df.head()

,idVotacao,deputado_siglaPartido,voto
0,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
23,0,0,0


## Criação de nova tabela sumarizada

### Criação de grupos por idVotacao e novo DataFrame com números de votos por partido por votação

In [9]:
groups = df.groupby('idVotacao')

newmatriz = []

for group in range(len(groups)):
    a = groups.get_group(group)
    for partido in range(len(uniques_deputado_siglaPartido)):
        idvotacao = a["idVotacao"].iloc[0]
        countnao =  a[(a["deputado_siglaPartido"] == partido) & (a["voto"] == 0)].count()[0]
        countsim = a[(a["deputado_siglaPartido"] == partido) & (a["voto"] == 1)].count()[0]
        linha = [idvotacao, partido, countsim, countnao]
        newmatriz.append(linha)

array = np.array(newmatriz)

newData = pd.DataFrame(array, columns=["idVotacao", "idPartido", "sim", "nao"])

### Verificação de novo DataFrame criado

In [10]:
newData.describe()

,idVotacao,idPartido,sim,nao
count,24856.000000,24856.000000,24856.000000,24856.000000
mean,477.500000,12.500000,6.988011,5.946250
std,275.978829,7.500151,11.156332,10.324765
min,0.000000,0.000000,0.000000,0.000000
25%,238.750000,6.000000,0.000000,0.000000
50%,477.500000,12.500000,1.000000,1.000000
75%,716.250000,19.000000,8.000000,7.000000
max,955.000000,25.000000,53.000000,53.000000


### Exportação do novo DataFrame para um arquivo

In [11]:
newData.to_csv("newData.csv", index=False)

### Leitura do novo DataFrame

In [12]:
df_clean = pd.read_csv("newData.csv")

df_clean.head()

,idVotacao,idPartido,sim,nao
0,0,0,0,5
1,0,1,0,4
2,0,2,1,0
3,0,3,0,4
4,0,4,0,2
